In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html
!pip install pytorch_lightning
!pip install wandb
#!wandb login

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 220.6 MB 36 kB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 281 kB 91.7 MB/s 
     |████████████████████████████████| 45 kB 4.0 MB/s 
     |████████████████████████████████| 11.1 MB 68.6 MB/s 
     |████████████████████████████████| 103 kB 85.0 MB/s 
     |████████████████████████████████| 109 kB 84.4 MB/s 
     |████████████████████████████████| 596 kB 87.0 MB/s 
     |████████████████████████████████| 51 kB 781 kB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 3.1 MB 56.7 MB/s 
     |████████████████████████████████| 546 kB 32.8 MB/s 
     |████████████████████████████████| 90 kB 10.3 MB/s 
     |████████████████████████████████| 100 kB 12.9 MB/s 
     |██████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 10.6 MB/s 
     |████████████████████████████████| 419 kB 78.7 MB/s 
     |████████████████████████████████| 140 kB 91.5 MB/s 
     |████████████████████████████████| 1.1 MB 5.9 MB/s 
     |████████████████████████████████| 271 kB 74.0 MB/s 
     |████████████████████████████████| 144 kB 86.0 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 18.6 MB/s 
     |████████████████████████████████| 181 kB 74.6 MB/s 
     |████████████████████████████████| 145 kB 76.7 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=c7a631ae6566e3cdf4af82c156819b251976defd4cc419788999eff58af06a2c
  Stored in di

In [2]:
%ls
%load_ext autoreload
%autoreload 2

drive/  sample_data/


In [3]:

%cd drive
%cd MyDrive
%cd SIG-VAE-GIN-and-fMRI
%cd SIG-VAE-GIN-and-fMRI

from sigvaemodel_transformer import *
from loss import *
from Dataset import *


/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI/SIG-VAE-GIN-and-fMRI


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [4]:
%reload_ext autoreload
from sigvaemodel_transformer import *
from loss import loss as loss_function
from Dataset import *
#from graph_transformer_edge_layer import *
from graph_transformer_layer import *
from mlp_readout_layer import *
#from loss import get_rec

In [5]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import dgl.data
import copy
import networkx as nx
import scipy.sparse as sp
from sklearn.metrics import average_precision_score, roc_curve, precision_recall_curve, roc_auc_score, mean_squared_error
import sigvaemodel_transformer
import loss
import Dataset
import time
import wandb
import torch.optim as optim
import gc
import random
from pytorch_lightning.loggers import WandbLogger
from tqdm import tqdm
#from torchviz import make_dot
device=torch.device('cuda')
#fMRI_load=torch.load('HCP_REST1_LR_schaefer400_sub19.pth', map_location=device)


In [6]:
'''
#os.chdir('..')
#%cd SIG-VAE-GIN-and-fMRI
!pwd
#print(os.listdir('/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI/SIG-VAE-GIN-and-fMRI/fMRI'))
#prepare_HCPRest_timeseries(atlas='schaefer400_sub19')
dataset=DatasetHCPRest(atlas='schaefer400_sub19', target_feature='Gender', k_fold=None, session='REST1', phase_encoding='LR')
dataloader=DataLoader(dataset, batch_size=1, shuffle=False)
batch_iterator=iter(dataloader)
test=next(batch_iterator)
print("################")
print(test)
print(len(test))
print(len(test.values()))
print((test['id']))
print(test['timeseries'].shape)
print(len(test['label']))
'''

'\n#os.chdir(\'..\')\n#%cd SIG-VAE-GIN-and-fMRI\n!pwd\n#print(os.listdir(\'/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI/SIG-VAE-GIN-and-fMRI/fMRI\'))\n#prepare_HCPRest_timeseries(atlas=\'schaefer400_sub19\')\ndataset=DatasetHCPRest(atlas=\'schaefer400_sub19\', target_feature=\'Gender\', k_fold=None, session=\'REST1\', phase_encoding=\'LR\')\ndataloader=DataLoader(dataset, batch_size=1, shuffle=False)\nbatch_iterator=iter(dataloader)\ntest=next(batch_iterator)\nprint("################")\nprint(test)\nprint(len(test))\nprint(len(test.values()))\nprint((test[\'id\']))\nprint(test[\'timeseries\'].shape)\nprint(len(test[\'label\']))\n'

In [7]:
def edges_to_lists(headMatrix, tailMatrix):
    results=[]
    assert len(headMatrix)==len(tailMatrix)
    hlength=len(headMatrix)
    for i in range(hlength):
        results.append([headMatrix[i], tailMatrix[i]])
    return results

In [8]:
def laplacian_positional_encoding(g, pos_enc_dim):
    """
        Graph positional encoding v/ Laplacian eigenvectors
    """

    # Laplacian
    A = g.adjacency_matrix_scipy(return_edge_ids=False).astype(float)
    N = sp.diags(dgl.backend.asnumpy(g.in_degrees()).clip(1) ** -0.5, dtype=float)
    L = sp.eye(g.number_of_nodes()) - N * A * N

    # Eigenvectors with scipy
    #EigVal, EigVec = sp.linalg.eigs(L, k=pos_enc_dim+1, which='SR')
    EigVal, EigVec = sp.linalg.eigs(L, k=pos_enc_dim+1, which='SR', tol=1e-2) # for 40 PEs
    EigVec = EigVec[:, EigVal.argsort()] # increasing order
    g.ndata['lap_pos_enc'] = torch.from_numpy(EigVec[:,1:pos_enc_dim+1]).float() 

    return g

In [9]:
def step(model, adj, adj_label, feat, input_graph, batch_graph, epoch, K, J, weight, norm, dropout, optimizer, scheduler, device):
  #global loss
    if optimizer is None:
        with torch.no_grad():
          model.eval()
        WU=1
    else:
        model.train()
        optimizer.zero_grad()
        #WU=torch.min(torch.Tensor([epoch/200., 1.]))
        WU=1
        
    generated_prob, mu, sigma, latent_representation, Z, epsilon, rk=model.forward(adj.to(device), feat.to(device), input_graph, batch_graph)
    loss=loss_function(generated_prob, adj_label, mu, sigma, Z, epsilon, latent_representation, K, J, WU, norm, weight, dropout, device=device)
    
    
    if optimizer is not None:
        loss.backward()
        optimizer.step()
        if scheduler is not None:
          scheduler.step()
    del mu
    del sigma
    del epsilon
    return latent_representation, generated_prob, loss, Z, rk

In [10]:
def get_auc(embedding, adj_orig, rk, decoder_type='bp'):

    J=embedding.shape[0]
    rk_expand=torch.tile(torch.diag(rk), (J, 1, 1))
    X=torch.transpose(embedding, 1, 2)
    X=torch.bmm(rk_expand, X)
    X=torch.bmm(embedding, X)
    X=torch.clamp(X, min=float('-inf'), max=10)
    
    adj_rec=1-torch.exp(-torch.exp(X)).detach().cpu().numpy()
    adj_orig_flatten=torch.flatten(adj_orig).detach().cpu().numpy() #flatten to calculate AUROC
    roc=torch.Tensor([roc_auc_score(adj_orig_flatten, adj_recon.flatten()) for adj_recon in np.vsplit(adj_rec, embedding.shape[0])]).mean()
    avg_precision=torch.Tensor([average_precision_score(adj_orig_flatten, adj_recon.flatten()) for adj_recon in np.vsplit(adj_rec, embedding.shape[0])]).mean()
    
    
    del adj_rec
    del adj_orig_flatten
    del X
    del rk_expand

    return roc, avg_precision #, precision_mean, recall_mean, fpr_mean, tpr_mean

In [11]:
def get_mse(embedding, adj_orig, rk, decoder_type='bp'):
    
    J=embedding.shape[0]
    rk_expand=torch.tile(torch.diag(rk), (J, 1, 1))
    X=torch.transpose(embedding, 1, 2)
    X=torch.bmm(rk_expand, X)
    X=torch.bmm(embedding, X)
    X=torch.clamp(X, min=float('-inf'), max=10)

    adj_rec=1-torch.exp(-torch.exp(X)).detach().cpu().numpy()
    adj_orig_flatten=torch.flatten(adj_orig).detach().cpu().numpy() #flatten to calculate AUROC
    mse=torch.Tensor([mean_squared_error(adj_orig_flatten, adj_recon.flatten()) for adj_recon in np.vsplit(adj_rec, embedding.shape[0])]).mean()

    del adj_rec
    del adj_orig_flatten
    del X
    del rk_expand

    return mse

In [12]:
def corrcoef(x):
    #from STAGIN
    mean_x = torch.mean(x, 1, keepdim=True)
    xm = x.sub(mean_x.expand_as(x))
    c = xm.mm(xm.t())
    c = c / (x.size(1) - 1)
    d = torch.diag(c)
    stddev = torch.pow(d, 0.5)
    c = c.div(stddev.expand_as(c))
    c = c.div(stddev.expand_as(c).t())
    c = torch.clamp(c, -1.0, 1.0)
    return c

In [13]:
def generateFC(argTensor, self_loop=True):
  #shape of argTensor : (number of ROIs=419, number of timepoints=1200)
  fc=corrcoef(argTensor)
  if not self_loop:
      fc-=torch.eye(fc.shape[0])
  fc_matrix=fc
  #print(fc_matrix.shape)
  return fc_matrix

In [14]:
def graphFC(n, pos_enc_dim, quantile, fc_matrix):
    '''
    using weighted FC as adjacency matrix
    '''
    threshold=torch.quantile(fc_matrix, quantile, dim=None)
    adj=torch.where(fc_matrix>=threshold, 1, 0)
    #adj=fc_matrix
    adj_u, adj_v=torch.where(adj!=0)
    #adj_u=adj
    #adj_v=adj
    fc_graph=dgl.graph((torch.hstack((adj_u, adj_v)), torch.hstack((adj_v, adj_u))), num_nodes=n)
    featureMatrix=torch.eye(n)
    fc_graph.ndata['feat']=featureMatrix
    fc_graph.edata['FC']=torch.ones(torch.count_nonzero(adj)*2)
    fc_graph=laplacian_positional_encoding(fc_graph, pos_enc_dim)
    featureMatrix3D=featureMatrix.unsqueeze(0)
    return fc_graph, adj, featureMatrix3D

In [15]:
def train(num_epochs, target_feature, k_fold, dataset, targetdir, seed, quantile, learning_rate, num_ROI=419, graphbatch_size=25, minibatch_size=1):
    
    # make directories
    os.makedirs(os.path.join(targetdir, 'model'), exist_ok=True)
    os.makedirs(os.path.join(targetdir, 'summary'), exist_ok=True)
    
    experiment_name=f'{num_epochs}_{target_feature}_{quantile}_{learning_rate}_train'
    # set seed and device
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        device = torch.device("cuda")
        torch.cuda.manual_seed_all(seed)
    else:
        device = torch.device("cpu")

    # define dataset
    if dataset=='rest': dataset = DatasetHCPRest(atlas='schaefer400_sub19', target_feature=target_feature, k_fold=k_fold, session='REST1', phase_encoding='LR')
    else: raise
    dataloader = DataLoader(dataset, batch_size=minibatch_size, shuffle=False, num_workers=4, pin_memory=True)

    # resume checkpoint if file exists
    if os.path.isfile(os.path.join(targetdir, 'checkpoint.pth')):
        print('resuming checkpoint experiment')
        checkpoint = torch.load(os.path.join(targetdir, 'checkpoint.pth'), map_location=device)
    else:
        checkpoint = {
            'fold': 0,
            'epoch': 0,
            'model': None,
            'optimizer': None,
            'scheduler': None}

    featureMatrix=(torch.eye(num_ROI)).unsqueeze(0)
    print("$$$$")
    print(featureMatrix.shape)
    # start experiment
    for k in range(checkpoint['fold'], k_fold):
        # make directories per fold
        os.makedirs(os.path.join(targetdir, 'model', str(k)), exist_ok=True)

        # set dataloader
        dataset.set_fold(k, train=True)

        # define model
        model=SIGVAE_GIN_Transformer(
                Lu=1, 
                Lmu=1, 
                Lsigma=1, 
                input_dim=num_ROI, #number of ROIs in .pth file
                output_dim_u=[32], 
                output_dim_mu=[32], 
                output_dim_sigma=[32],
                K=5,
                J=8,
                lap_pos_enc_dim=4,
                device=device,
                noise_dim=64,
                decoder_type="bp",
                activation=torch.nn.functional.relu,
                dropout=0)
        model.to(device)
        if checkpoint['model'] is not None: model.load_state_dict(checkpoint['model'])

        # define optimizer and learning rate scheduler
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        scheduler=optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0, verbose=False)
        if checkpoint['optimizer'] is not None: optimizer.load_state_dict(checkpoint['optimizer'])
        if checkpoint['scheduler'] is not None: scheduler.load_state_dict(checkpoint['scheduler'])
        '''
        # define logging objects
        summary_writer = SummaryWriter(os.path.join(argv.targetdir, 'summary', str(k), 'train'), )
        summary_writer_val = SummaryWriter(os.path.join(argv.targetdir, 'summary', str(k), 'val'), )
        logger = util.logger.LoggerSTAGIN(argv.k_fold, dataset.num_classes)
        '''
        '''
        wandb.init(project="SIG-VAE-GIN-and-fMRI", entity="ytrewq271828")
        wandb.config = {
            "learning_rate": learning_rate,
            "epochs": num_epochs,
            "graph_batch_size": graphbatch_size,
            }
        wandb.run.name = f"{experiment_name}_{wandb.run.id}"
        wandb.run.save()
        logger = WandbLogger(project="SIG-VAE-GIN-and-fMRI", name=experiment_name)
        ''' 
        # start training
        for epoch in range(checkpoint['epoch'], num_epochs):
            dataset.set_fold(k, train=True)
            loss_accumulate = 0.0
            for i, x in enumerate(tqdm(dataloader, ncols=60, desc=f'k:{k} e:{epoch}')):
                t=time.time()
                # process input data
                mean_timeseries=x['timeseries'].squeeze(0)
                N=mean_timeseries.shape[0]
                fc_matrix=generateFC(mean_timeseries, self_loop=True)
                graph, adj, feat=graphFC(N, quantile=quantile, pos_enc_dim=4, fc_matrix=fc_matrix)
                graph=graph.to(device)
                adj_label=adj+torch.eye(N)
                '''
                dyn_a, sampling_points = util.bold.process_dynamic_fc(x['timeseries'], argv.window_size, argv.window_stride, argv.dynamic_length)
                sampling_endpoints = [p+argv.window_size for p in sampling_points]
                if i==0: dyn_v = repeat(torch.eye(dataset.num_nodes), 'n1 n2 -> b t n1 n2', t=len(sampling_points), b=argv.minibatch_size)
                if len(dyn_a) < argv.minibatch_size: dyn_v = dyn_v[:len(dyn_a)]
                t = x['timeseries'].permute(1,0,2)
                label = x['label']
                '''
                #N=adj.shape[0]
                batch_graph=[]
                for i in range(graphbatch_size):
                  train_graph_copy=copy.deepcopy(graph)
                  batch_graph.append(train_graph_copy)
                weight=torch.tensor([float(N * N - adj.sum()) / adj.sum()]).to(device)
                norm=N*N/float((N*N-adj.sum())*2)
                latent_representation, generated_prob, loss_train, z_proc, rk=step(
                    model=model,
                    adj=adj, 
                    adj_label=adj_label, 
                    feat=feat, 
                    input_graph=graph, 
                    batch_graph=batch_graph, 
                    epoch=epoch, 
                    K=5, 
                    J=8,
                    weight=weight, 
                    norm=norm, 
                    dropout=0, 
                    optimizer=optimizer, 
                    scheduler=scheduler, 
                    device=device)
                #loss_accumulate += loss.detach().cpu().numpy()
                loss_accumulate += loss_train

                roc, avg_precision=get_auc(z_proc, adj, rk, decoder_type="bp")
                #mse=get_mse(z_proc, adj, rk, decoder_type="bp")
                print("\nEpoch : ", '%d' %(epoch+1), "fold : ", '%d' %(k), "loss=", "%f" %(loss_train.item()),
                      "time=", "%f" %(time.time()-t), "roc=", "%f" %(roc), "average_precision", "%f" %(avg_precision))
                #print("\nEpoch : ", '%d' %(epoch+1), "fold : ", '%d' %(k), "loss=", "%f" %(loss_train.item()),
                #      "time=", "%f" %(time.time()-t), "mean-squared error : ", '%f' %(mse))
            # save checkpoint
            torch.save({
                'fold': k,
                'epoch': epoch+1,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict()},
                os.path.join(targetdir, 'checkpoint.pth'))

        # finalize fold
        torch.save(model.state_dict(), os.path.join(targetdir, 'model', str(k), 'model.pth'))
        checkpoint.update({'epoch': 0, 'model': None, 'optimizer': None, 'scheduler': None})
    '''
    wandb.finish()
    '''
    os.remove(os.path.join(targetdir, 'checkpoint.pth'))



In [16]:
def test(num_epochs, target_feature, k_fold, dataset, targetdir, seed, quantile, learning_rate, num_ROI=419, graphbatch_size=13, minibatch_size=1):
    # make directories
    os.makedirs(os.path.join(targetdir, 'model'), exist_ok=True)
    os.makedirs(os.path.join(targetdir, 'summary'), exist_ok=True)

    experiment_name=f'{num_epochs}_{target_feature}_{quantile}_{learning_rate}_test'
    # set seed and device
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        device = torch.device("cuda")
        torch.cuda.manual_seed_all(seed)
    else:
        device = torch.device("cpu")

    # define dataset
    if dataset=='rest': dataset = DatasetHCPRest(atlas='schaefer400_sub19', target_feature=target_feature, k_fold=k_fold, session='REST1', phase_encoding='LR')
    else: raise
    dataloader = DataLoader(dataset, batch_size=minibatch_size, shuffle=False, num_workers=4, pin_memory=True)

    featureMatrix=(torch.eye(num_ROI)).unsqueeze(0)
    # start experiment
    for k in range(k_fold):
        # make directories per fold
        os.makedirs(os.path.join(targetdir, 'model', str(k)), exist_ok=True)

        # set dataloader
        dataset.set_fold(k, train=True)

        # define model
        model=SIGVAE_GIN_Transformer(
                Lu=1, 
                Lmu=1, 
                Lsigma=1, 
                input_dim=num_ROI, #number of ROIs in .pth file
                output_dim_u=[32], 
                output_dim_mu=[32], 
                output_dim_sigma=[32],
                K=5,
                J=8,
                lap_pos_enc_dim=4,
                device=device,
                noise_dim=64,
                decoder_type="bp",
                activation=torch.nn.functional.relu,
                dropout=0)
        model.to(device)
        '''
        wandb.init(project="SIG-VAE-GIN-and-fMRI", entity="ytrewq271828")
        wandb.config = {
            "learning_rate": learning_rate,
            "epochs": num_epochs,
            "graph_batch_size": graphbatch_size,
            }
        wandb.run.name = f"{experiment_name}_{wandb.run.id}"
        wandb.run.save()
        logger = WandbLogger(project="SIG-VAE-GIN-and-fMRI", name=experiment_name)
        '''
        # start testing
        dataset.set_fold(k, train=True)
        loss_accumulate = 0.0
        for i, x in enumerate(tqdm(dataloader, ncols=60, desc=f'k:{k}')):
          with torch.no_grad():
            # process input data
            mean_timeseries=torch.mean(x['timeseries'].squeeze(0), dim=0) #len=419
            N=mean_timeseries.shape[0]
            fc_matrix=generateFC(mean_timeseries, self_loop=True)
            graph, adj, feat=graphFC(N, quantile=quantile, pos_enc_dim=4, fc_matrix=fc_matrix)
            graph=graph.to(device)
            adj_label=adj+torch.eye(N)
           

            batch_graph=[]
            for i in range(graphbatch_size):
              train_graph_copy=copy.deepcopy(graph)
              batch_graph.append(train_graph_copy)
            weight=torch.tensor([float(N * N - adj.sum()) / adj.sum()]).to(device)
            norm=N*N/float((N*N-adj.sum())*2)
            latent_representation, generated_prob, loss_test, z_proc, rk=step(
                model=model,
                adj=adj, 
                adj_label=adj_label, 
                feat=feat, 
                input_graph=graph, 
                batch_graph=batch_graph, 
                epoch=0, #have no role in step() while testing 
                K=5, 
                J=8,
                weight=weight, 
                norm=norm, 
                dropout=0, 
                optimizer=None, 
                scheduler=None, 
                device=device)
            #loss_accumulate += loss.detach().cpu().numpy()
           
            roc, avg_pre= get_auc(z_proc, adj, rk, decoder_type="bp")
            if roc>max_test_auc:
              max_test_auc=roc
            if avg_pre>max_test_ap:
              max_test_ap=avg_pre
            #mse=get_mse(z_proc, adj, rk, decoder_type="bp")
            rslt = "Test ROC score: %f" %(roc)
            rslt2="Test Precision score : %f" %(avg_pre)
            #rslt="Test mean-squared error : %f" %(mse)
            print("\n", rslt, " ", rslt2, "\n")
        # finalize fold
        torch.save(model.state_dict(), os.path.join(targetdir, 'model', str(k), 'model.pth'))
    '''
    wandb.finish()
    '''
    os.remove(os.path.join(targetdir, 'checkpoint.pth'))


In [17]:
#main
import warnings
if __name__=='__main__':
  warnings.filterwarnings(action='ignore')

  num_epochs=10
  target_feature='Gender'
  k_fold=5
  dataset='rest'
  target_dir='/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI/SIG-VAE-GIN-and-fMRI/experiments/BinaryFC'
  seed=5
  quantile=0.3
  learning_rate=0.0005
  num_ROI=419
  graphbatch_size=13
  minibatch_size=1

  train(num_epochs, target_feature, k_fold, dataset, target_dir, seed, quantile, learning_rate, num_ROI, graphbatch_size, minibatch_size)
  print("train done!")
  test(num_epochs, target_feature, k_fold, dataset, target_dir, seed, quantile, learning_rate, num_ROI, graphbatch_size, minibatch_size)


/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI/SIG-VAE-GIN-and-fMRI/fMRI/HCP_REST1_LR_schaefer400_sub19.pth


KeyboardInterrupt: ignored